In [ ]:
!pip install folium

In [2]:
import folium

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("dataset/dataset_tcc_final.csv")
df.head()

,estacao,dataNotif,ocupacaoSuspeitoCli,ocupacaoSuspeitoUti,ocupacaoConfirmadoCli,ocupacaoConfirmadoUti,saidaSuspeitaObitos,saidaSuspeitaAltas,saidaConfirmadaObitos,saidaConfirmadaAltas,lat,long,no_fantasia,tp_gestao,no_bairro
0,VERAO,04/01/2020,113.0,70.0,34.0,25.0,0,1,0,3,-23.57005,-46.64571,HOSPITAL SANTA CATARINA,M,BELA VISTA
1,VERAO,07/02/2020,13.0,0.0,0.0,0.0,0,0,0,0,-23.53166,-46.62066,HOSP NSRA DO PARI,M,CANINDE
2,VERAO,20/03/2020,5.0,0.0,0.0,0.0,0,0,0,0,-23.56392,-46.59042,CEMA HOSP ESPEC,M,MOOCA
3,OUTONO,01/04/2020,91.0,44.0,0.0,3.0,0,0,0,0,-23.54198,-46.63503,HOSP MUN GILSON DE CASSIA MARQUES DE CARVALHO,M,VILA MASCOTE
4,OUTONO,01/04/2020,56.0,27.0,29.0,10.0,0,0,3,10,-23.69182,-46.77556,HOSP MUN M BOI MIRIM,M,JARDIM ANGELA


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39894 entries, 0 to 39893
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   estacao                39894 non-null  object 
 1   dataNotif              39894 non-null  object 
 2   ocupacaoSuspeitoCli    34732 non-null  float64
 3   ocupacaoSuspeitoUti    34732 non-null  float64
 4   ocupacaoConfirmadoCli  34732 non-null  float64
 5   ocupacaoConfirmadoUti  34732 non-null  float64
 6   saidaSuspeitaObitos    39894 non-null  int64  
 7   saidaSuspeitaAltas     39894 non-null  int64  
 8   saidaConfirmadaObitos  39894 non-null  int64  
 9   saidaConfirmadaAltas   39894 non-null  int64  
 10  lat                    39894 non-null  float64
 11  long                   39894 non-null  float64
 12  no_fantasia            39894 non-null  object 
 13  tp_gestao              39894 non-null  object 
 14  no_bairro              39894 non-null  object 
dtypes:

In [6]:
import json
import requests
import urllib.request
from branca.element import Template, MacroElement

In [7]:
import folium
mapa = folium.Map(location=[-23.5489,-46.6388])
mapa

In [8]:
cidade = ['São Paulo']
num_data = [0]
sample = pd.DataFrame({'cidade':cidade, 'data':num_data})

In [9]:
sp_geojson = "https://raw.githubusercontent.com/tbrugz/geodata-br/master/geojson/geojs-35-mun.json"
 
with urllib.request.urlopen(sp_geojson) as url:
    geo_data = json.loads(url.read().decode())

In [10]:
folium.Choropleth(
    geo_data=geo_data,
    name='choropleth',
    data=sample,
    columns=['cidade', 'data'],
    key_on='feature.properties.name',
    fill_color='YlGn',
    fill_opacity=0.4,
    line_opacity=0.2,
    legend_name='Cidade'
).add_to(mapa)

In [15]:
df_cnes = df.groupby(['lat','long','tp_gestao']).sum().reset_index()
df_cnes_resumo = df_cnes[['lat','long','tp_gestao']]
df_cnes_resumo


,lat,long,tp_gestao
0,-23.763270,-46.706010,M
1,-23.729540,-46.690770,E
2,-23.728929,-46.711566,M
3,-23.707090,-46.701960,E
4,-23.706146,-46.742127,M
...,...,...,...
114,-23.466540,-46.584000,M
115,-23.460600,-46.582460,M
116,-23.441450,-46.799890,M
117,-23.438280,-46.717280,E


In [16]:
for lat, lng, tp_gestao in zip(df_cnes_resumo.lat, df_cnes_resumo.long, df_cnes_resumo.tp_gestao):
    if tp_gestao == "M":
        folium.Marker([lat,lng],icon=folium.Icon(color='orange', prefix='fa', icon="ambulance")).add_to(mapa)
    else:
        folium.Marker([lat,lng],icon=folium.Icon(color='blue', prefix='fa', icon="ambulance")).add_to(mapa)

In [17]:
template = """
{% macro html(this, kwargs) %}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <title>jQuery UI Draggable - Default functionality</title>
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">

  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>
  
  <script>
  $( function() {
    $( "#maplegend" ).draggable({
                    start: function (event, ui) {
                        $(this).css({
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        });
                    }
                });
});

  </script>
</head>
<body>

 
<div id='maplegend' class='maplegend' 
    style='position: absolute; z-index:9999; border:2px solid grey; background-color:rgba(255, 255, 255, 0.8);
     border-radius:6px; padding: 10px; font-size:14px; right: 20px; bottom: 20px;'>
     
<div class='legend-title'>Legenda</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    <li><span style='background:#1E90FF;opacity:0.7;'></span>Hospital Estadual</li>
    <li><span style='background:#FF8C00;opacity:0.7;'></span>Hospital Municipal</li>
  </ul>
</div>
</div>
 
</body>
</html>

<style type='text/css'>
  .maplegend .legend-title {
    text-align: left;
    margin-bottom: 5px;
    font-weight: bold;
    font-size: 90%;
    }
  .maplegend .legend-scale ul {
    margin: 0;
    margin-bottom: 5px;
    padding: 0;
    float: left;
    list-style: none;
    }
  .maplegend .legend-scale ul li {
    font-size: 80%;
    list-style: none;
    margin-left: 0;
    line-height: 18px;
    margin-bottom: 2px;
    }
  .maplegend ul.legend-labels li span {
    display: block;
    float: left;
    height: 16px;
    width: 30px;
    margin-right: 5px;
    margin-left: 0;
    border: 1px solid #999;
    }
  .maplegend .legend-source {
    font-size: 80%;
    color: #777;
    clear: both;
    }
  .maplegend a {
    color: #777;
    }
</style>
{% endmacro %}"""

macro = MacroElement()
macro._template = Template(template)

mapa.get_root().add_child(macro)

mapa